<a href="https://colab.research.google.com/github/ManishBarath/Cats-and-Dogs--Audio-Classification/blob/main/cats_dogs_audio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("mmoreaux/audio-cats-and-dogs")

print("Path to dataset files:", path)

100%|██████████| 97.0M/97.0M [00:04<00:00, 24.3MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/mmoreaux/audio-cats-and-dogs/versions/5


In [ ]:
import os
import torchaudio
import librosa
import numpy as np

In [ ]:
train_dataset_path = "/content/cats_dogs/train"
test_dataset_path = "/content/cats_dogs"


In [ ]:
labels = {
    "cat" : 0,
    "dog" :1
}

In [ ]:
train_x = []
train_y = []

In [ ]:
for animal in os.listdir(train_dataset_path):
    for file in os.listdir(os.path.join(train_dataset_path, animal)):
        train_x.append(os.path.join(train_dataset_path, animal, file))
        train_y.append(labels[animal])

FileNotFoundError: [Errno 2] No such file or directory: '/content/cats_dogs/train'

In [ ]:
print(train_x)
print(len(train_x))

In [ ]:
from sklearn.svm import SVC

In [ ]:
model = SVC()

In [ ]:
extracted_x = []

In [ ]:
def extract_feature(path):
       try:
           waveform, sample_rate = torchaudio.load(path)
           f = librosa.feature.mfcc(y=waveform.numpy()[0], sr=sample_rate, n_mfcc=500)  # Access the first channel of the waveform
           return np.mean(f.T, axis=0)  # Return mean MFCCs
       except Exception as e:
           print(f"Error processing {path}: {e}")
           return None  # Return None if there is an error

In [ ]:
for path in train_x:
  extracted_x.append(extract_feature(path))

In [ ]:
print(extracted_x[0])

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
extracted_x_scaled = scaler.fit_transform(extracted_x)

In [ ]:
print(extracted_x_scaled[0])

In [ ]:
model = model.fit(extracted_x_scaled , train_y)

In [ ]:
test_x = []
test_y = []

In [ ]:
for animal in os.listdir(test_dataset_path):
      if animal.endswith(".wav"):
        test_x.append(os.path.join(test_dataset_path, animal))
        if animal.startswith("cat"):
          test_y.append(0)
        else:
          test_y.append(1)

In [ ]:
print(test_x)

print(test_y)

In [ ]:

extracted_test_x = []
for path in test_x:
    features = extract_feature(path)
    if features is not None:
        extracted_test_x.append(features)
    else:
        print(f"Skipping {path} due to error")

In [ ]:
print(extracted_test_x[0])

In [ ]:
extracted_x_scaled = scaler.fit_transform(extracted_test_x)

In [ ]:
print(extracted_x_scaled[0])

In [ ]:
predicted = model.predict(extracted_x_scaled)

In [ ]:
cnt = 0
for i in range(len(predicted)):
  if predicted[i] == test_y[i]:
    cnt += 1

In [ ]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(test_y, predicted)
print(f"Accuracy: {accuracy}")
